In [ ]:
# installing the libraries that we need
!pip install -q torch transformers sentence-transformers wandb scikit-learn rapidfuzz


In [ ]:
!pip install rapidfuzz

In [ ]:
# Standard Imports
import os
import re
import json
import math
import textwrap
from typing import List, Dict, Tuple

#Numpy/Pandas for data wrangling
import numpy as np
import pandas as pd

#HF sentence-transformers for sentence/phrase embeddings
from sentence_transformers import SentenceTransformer, util

#Fuzzy string matching for messy text
from rapidfuzz import fuzz, process

#Using this to generate simple tables/reports in the notebook
from IPython.display import display, HTML


In [ ]:
import wandb
wandb.login()

wandb.init(
    project="jd-resume-tracker",
    name="baseline-hf-embeddings",
    config={
        "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
        "skill_vocab_version": "v0.1",
        "similarity_metric": "cosine",
        "use_fuzzy_fallback": True
    }
)

In [ ]:
#Example job description text(short)
job_description_text="""We are hiring a Machine Learning Engineer with strong experience in Python, PyTorch,
TensorFlow, and AWS. You should know data pipelines (Airflow), SQL, Spark, Docker,
Kubernetes, Git, and ML experiment tracking (Weights & Biases or MLflow).
NLP experience with Hugging Face is a plus.
"""

#Example resume text(short)
resume_text="""Sophomore EE with CS/AI focus. Projects in NLP and Transformers using Hugging Face.
Comfortable with Python, Pandas, NumPy, scikit-learn, TensorFlow. Basic PyTorch.
Experience with Git/GitHub, Docker, SQL. Learning Airflow and Spark. Used Weights & Biases.
Some AWS exposure.
"""

In [ ]:
# A small practical skills vocabualry to start.
skill_vocab=[
     # Languages & Core
    "Python", "SQL", "Java", "C++",
    # ML/DL
    "PyTorch", "TensorFlow", "scikit-learn", "Hugging Face", "Transformers",
    # Data
    "Pandas", "NumPy", "Spark", "Airflow",
    # DevOps/Platform
    "Docker", "Kubernetes", "Git", "GitHub",
    # Tracking/Experimentation
    "Weights & Biases", "W&B", "MLflow",
    # Cloud
    "AWS", "SageMaker", "Lambda", "API Gateway", "S3", "Comprehend", "Bedrock"
]



In [ ]:
# Correct model name
embedding_model_name = wandb.config.get("embedding_model", "sentence-transformers/all-MiniLM-L6-v2")

# Load the SentenceTransformer model
embedder = SentenceTransformer(embedding_model_name)  # downloads from HF if not cached


In [ ]:
def normalize_text(t: str) -> str:
    """
    Light normalization to reduce noise:
    - lowercasing
    - trim spaces
    - collapse multiple spaces
    """
    # Convert to lowercase for case-insensitive matching
    t = t.lower()
    # Strip leading/trailing whitespace
    t = t.strip()
    # Collapse multiple spaces/newlines into single spaces
    t = re.sub(r"\s+", " ", t)
    return t

def extract_candidate_phrases(t: str, min_len=2, max_len=5) -> List[str]:
    """
    Very simple phrase candidate extraction:
    - Split text into tokens by non-letter chars
    - Build n-grams (2 to 5 words) + unigrams that look like skills
    This is a quick heuristic; we'll lean on embeddings + fuzzy later.
    """
    # Replace non-letters/numbers with spaces and split
    tokens = re.findall(r"[a-zA-Z0-9\+\.\-]+", t)
    # Collect unigrams that might be skills (e.g., 'python', 'pytorch')
    unigrams = [tok for tok in tokens if len(tok) >= 2]
    # Build simple n-grams (2..5 words) to catch multiword skills (e.g., 'hugging face')
    ngrams = []
    for n in range(min_len, max_len + 1):
        for i in range(len(tokens) - n + 1):
            ngrams.append(" ".join(tokens[i:i+n]))
    # Combine and deduplicate while keeping order
    seen = set()
    out = []
    for p in unigrams + ngrams:
        if p not in seen:
            seen.add(p)
            out.append(p)
    return out

def embed_texts(texts: List[str]) -> np.ndarray:
    """
    Compute embeddings for a list of strings using the SentenceTransformer embedder.
    Returns a NumPy array of shape (len(texts), dim).
    """
    # Encode returns a list/array of vectors; convert to numpy for convenience
    return np.array(embedder.encode(texts, normalize_embeddings=True))

def top_matches(
    queries: List[str],
    corpus: List[str],
    k: int = 5,
    use_fuzzy: bool = True,
    fuzzy_threshold: int = 90
) -> Dict[str, List[Tuple[str, float]]]:
    """
    For each query (candidate phrase), find top-k matches in the corpus (skill vocab).
    - Primary: cosine similarity on embeddings.
    - Fallback: fuzzy matching to catch spelling/format differences.
    Returns dict: query -> list of (matched_skill, score) tuples.
    """
    # Compute embeddings for both sides
    q_emb = embed_texts(queries)
    c_emb = embed_texts(corpus)

    # Cosine similarity matrix (queries x corpus)
    sim = np.matmul(q_emb, c_emb.T)  # normalized embeddings => dot product == cosine similarity

    results = {}
    for i, q in enumerate(queries):
        # Get top-k by cosine
        idxs = np.argsort(-sim[i])[:k]
        pairs = [(corpus[j], float(sim[i, j])) for j in idxs]

        # Optional fuzzy pass if best cosine is weak and we want to catch typos
        if use_fuzzy and (len(pairs) == 0 or pairs[0][1] < 0.5):
            # Use rapidfuzz to find likely literal matches (threshold adjustable)
            fuzzy = process.extract(q, corpus, scorer=fuzz.token_sort_ratio, limit=k)
            # Convert rapidfuzz (str, score, idx) -> (str, normalized_score)
            fuzzy = [(name, score / 100.0) for name, score, _ in fuzzy if score >= fuzzy_threshold]
            if fuzzy:
                pairs = fuzzy

        results[q] = pairs
    return results

def choose_skills(
    phrase_to_matches: Dict[str, List[Tuple[str, float]]],
    min_cosine: float = 0.6
) -> List[str]:
    """
    Convert phrase->matches into a final deduped skill list.
    - Keep matches with cosine >= min_cosine (or fuzzy normalized score >= min_cosine).
    - Deduplicate by best score.
    """
    best: Dict[str, float] = {}
    for phrase, matches in phrase_to_matches.items():
        for skill, score in matches:
            if score >= min_cosine:
                if skill not in best or score > best[skill]:
                    best[skill] = score
    # Sort skills by score (desc) for presentation
    return [s for s, _ in sorted(best.items(), key=lambda kv: -kv[1])]


In [ ]:
# Normalize inputs for clean processing
jd_norm = normalize_text(job_description_text)
cv_norm = normalize_text(resume_text)

# Extract crude candidate phrases from both
jd_candidates = extract_candidate_phrases(jd_norm)
cv_candidates = extract_candidate_phrases(cv_norm)

# Compute matches from candidates to our curated skill vocabulary
jd_phrase_matches = top_matches(jd_candidates, [s.lower() for s in skill_vocab], k=3)
cv_phrase_matches = top_matches(cv_candidates, [s.lower() for s in skill_vocab], k=3)

# Decide which vocab skills appear in each text
jd_skills = choose_skills(jd_phrase_matches, min_cosine=0.6)
cv_skills = choose_skills(cv_phrase_matches, min_cosine=0.6)

# Convert back to canonical casing by mapping lower->original
lower_to_original = {s.lower(): s for s in skill_vocab}
jd_skills = [lower_to_original[s] for s in jd_skills if s in lower_to_original]
cv_skills = [lower_to_original[s] for s in cv_skills if s in lower_to_original]

# Match logic: overlap and gaps
have = sorted(set(jd_skills).intersection(cv_skills))
missing = sorted(set(jd_skills) - set(cv_skills))
coverage = 0.0 if len(jd_skills) == 0 else len(have) / len(jd_skills)

# Log to W&B so we can compare runs later
wandb.log({
    "jd_skill_count": len(jd_skills),
    "cv_skill_count": len(cv_skills),
    "overlap_count": len(have),
    "missing_count": len(missing),
    "coverage": coverage
})

# Display a quick report
display(HTML(f"""
<h3>Quick Match Report</h3>
<p><b>JD skills:</b> {', '.join(jd_skills) if jd_skills else 'None'}</p>
<p><b>Resume skills:</b> {', '.join(cv_skills) if cv_skills else 'None'}</p>
<p><b>Overlap (you have):</b> {', '.join(have) if have else 'None'}</p>
<p><b>Missing (to improve):</b> {', '.join(missing) if missing else 'None'}</p>
<p><b>Match coverage:</b> {coverage:.2%}</p>
"""))
